# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [106]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


def get_area(team):
    for each in list(nhl_cities.index.values):
        if team in each:
            return nhl_cities.at[each, 'Metropolitan area']


nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
nhl_cities = nhl_cities.drop(['—', ''], axis=0)
nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0) # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))
out = []
for group, frame in nhl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])


def nhl_correlation(): 
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = out_df['Population (2016 est.)[8]']#.to_list()
    #print(population_by_region)
    win_loss_by_region = out_df['Ratio']#.to_list()
    nhl_total = out_df[["Population (2016 est.)[8]","Ratio"]]
    nhl_total=nhl_total.reset_index()
    nhl_total=nhl_total.rename(columns={"index":"Metropolitan area","Population (2016 est.)[8]":"population","Ratio" : "W/L"})

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return nhl_total #np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

def get_nhl_data():
    return out_df

nhl_correlation()

,Metropolitan area,population,W/L
0,Boston,4794447,0.714286
1,Buffalo,1132804,0.357143
2,Calgary,1392609,0.513889
3,Chicago,9512999,0.458333
4,Columbus,2041520,0.600000
5,Dallas–Fort Worth,7233323,0.567568
6,Denver,2853077,0.589041
7,Detroit,4297617,0.434783
8,Edmonton,1321426,0.473684
9,Las Vegas,2155664,0.680000


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [72]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    #print(nba_df)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #print(cities["NBA"])
    nba_df=nba_df[nba_df.year == 2018]
    nba_df = nba_df[["team", "W", "L"]]
    #print(nba_df.shape) #(30,3) 
    #print(nba_df)
    
    nba_df['team'] = nba_df.team.apply(lambda team: re.findall("([\w\s]*)\*", team)[0] if '*' in team else team)
    #print(nba_df.shape) #(30,11)
    #print(nba_df)
    nba_df['team'] = nba_df.team.apply(lambda team: re.findall("([\w\s]*)\s\(", team)[0] if '(' in team else team)
    #print(nba_df.shape) #(30,11)
    #print(nba_df)
    cities['NBA'] = cities.NBA.apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if '[' in team else team)
    #print(cities['MLB'].shape) # (51,)
    #print(cities[["Metropolitan area","NBA"]])
    nba_df["team"]=nba_df['team'].replace({"Los Angeles Lakers" : "Lakers",
                                            "Los Angeles Clippers" : "Lakers",
                                            "New York Knicks":"Knicks",   
                                            "Brooklyn Nets":"Knicks"})
    #print(nba_df['team'])

    nba_df['team_only'] = nba_df.team.apply(lambda x:x.rsplit(None,1)[-1])
    #print(nba_df.shape) #(30.4)
    #print(nba_df['team_only'])

    nba_df["team_only"]=nba_df['team_only'].replace({"Lakers" : "LakersClippers",
                                                    "Knicks":"KnicksNets",
                                                    "Blazers":"Trail Blazers"})
    #print(nba_df['team_only'])  #there are 2 lakers clippers and 2 knicks nets

    cities.rename(columns={"Population (2016 est.)[8]":"population", "NBA":"team_only"},inplace=True)
    #cities["team_only"]=cities['team_only'].replace({"Nets":"Knicks Nets"})
    #print(cities["team_only"])
    
    nba_df=nba_df[["team_only","W","L"]]
    #print(nba_df.shape) #(30,3)
    nba_df.W=nba_df.W.astype("float")
    nba_df.L=nba_df.L.astype("float")
    #print(nba_df.shape) #(30,3)
    nba_df=nba_df.groupby(by="team_only")[["W","L"]].mean()
    #print(nba_df.shape) #(28,2) + the index column

    #nba_df= nba_df.to_excel("groupedBy Teamonly.xlsx") #######################

    nba_df.reset_index(inplace=True)
    #print(nba_df) #(28,3)
    #print(nba_df)
    merged_df=pd.merge(nba_df,cities, how="inner",on="team_only")
    #print(merged_df.shape) #(28.8)

    merged_df.W=nba_df.W.astype("float")
    merged_df.L=nba_df.L.astype("float")
    merged_df['win_loss_ratio']=merged_df.W/(merged_df.W + merged_df.L)
    #print(merged_df['win_loss_ratio'].shape) #(28,)
    
    #raise NotImplementedError()
    
    population_by_region = pd.to_numeric(merged_df["population"],errors='coerce')# pass in metropolitan area population from cities
    #print("population_by_region :",population_by_region.shape) #(28,)
    win_loss_by_region = pd.to_numeric(merged_df["win_loss_ratio"],errors='coerce') # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    #print("win_loss_by_region ",win_loss_by_region.shape) #(28,)
    nba_total= merged_df[["Metropolitan area","population","win_loss_ratio"]]
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return nba_total #stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

,Metropolitan area,population,win_loss_ratio
0,Philadelphia,6070500,0.634146
1,Milwaukee,1572482,0.536585
2,Chicago,9512999,0.329268
3,Cleveland,2055612,0.609756
4,Boston,4794447,0.670732
5,Memphis,1342842,0.268293
6,Atlanta,5789700,0.292683
7,Miami–Fort Lauderdale,6066387,0.536585
8,Charlotte,2474314,0.439024
9,Salt Lake City,1186187,0.585366


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [84]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    mlb_df=mlb_df[mlb_df.year == 2018]
    mlb_df = mlb_df[["team", "W", "L"]]
    #print(mlb_df.shape) #(30,3) 
    #print(mlb_df)
    cities['MLB'] = cities.MLB.apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if '[' in team else team)
    #print(cities['MLB'].shape) # (51,)
    #print(cities[["Metropolitan area","MLB"]])
    mlb_df["team"]=mlb_df['team'].replace({"Boston Red Sox":"Boston RedSox",
                                           "Toronto Blue Jays":"Toronto BlueJays",
                                            "Los Angeles Angels" : "Dodgers",
                                            "Los Angeles Dodgers" : "Dodgers",
                                            "New York Yankees":"Mets",   
                                            "New York Mets":"Mets",
                                            "San Francisco Giants":"Athletics",
                                            "Oakland Athletics":"Athletics",
                                            "Chicago Cubs":"Cubs",
                                            "Chicago White Sox":"Cubs"})
    #print(mlb_df['team'])

    mlb_df['team_only'] = mlb_df.team.apply(lambda x:x.rsplit(None,1)[-1])
    #print(mlb_df.shape) #(30.4)
    #print(mlb_df['team_only'])

    mlb_df["team_only"]=mlb_df['team_only'].replace({ "RedSox":"Red Sox",
                                                    "BlueJays":"Blue Jays",
                                                    "Dodgers" : "DodgersAngels",
                                                    "Mets":"YankeesMets",
                                                    "Athletics":"GiantsAthletics",
                                                    "Cubs":"CubsWhite Sox"})
    #print(mlb_df['team_only'])

    cities.rename(columns={"Population (2016 est.)[8]":"population", "MLB":"team_only"},inplace=True)
    #print(cities["team_only"])

    cities["team_only"]=cities['team_only'].replace({"RedSox":"Red Sox",
                                                    "BlueJays":"Blue Jays",
                                                    "CubsWhiteSox":"Cubs White Sox"})
    #print(cities["team_only"])
    #print(cities.shape) #(51,6)
    #print(mlb_df)
    mlb_df=mlb_df[["team_only","W","L"]]
    mlb_df.W=mlb_df.W.astype("float")
    mlb_df.L=mlb_df.L.astype("float")
    #print(mlb_df.shape) #(30,3)
    mlb_df=mlb_df.groupby(by="team_only")[["W","L"]].mean()
    #print(mlb_df.shape) #(26,2) + the index column

    mlb_df.reset_index(inplace=True)
    #print(mlb_df.shape) #(26,3)
    #print(mlb_df)
    merged_df=pd.merge(mlb_df,cities, how="inner",on="team_only")
    #print(merged_df.shape)

    #print("according to Visual Studio Environment (26,8)shape: ",merged_df.shape) #(26,8) according to Visual Studio Environment

    merged_df.W=mlb_df.W.astype("float")
    merged_df.L=mlb_df.L.astype("float")

    merged_df['win_loss_ratio']=merged_df.W/(merged_df.W + merged_df.L)
    #print(merged_df['win_loss_ratio'].shape)
    
    #raise NotImplementedError()
    
    population_by_region = pd.to_numeric(merged_df["population"],errors='coerce')# pass in metropolitan area population from cities
    #print("population_by_region :",population_by_region.shape)
    win_loss_by_region = pd.to_numeric(merged_df["win_loss_ratio"],errors='coerce') # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    #print("win_loss_by_region ",win_loss_by_region.shape)
    mlb_total= merged_df[["Metropolitan area","population","win_loss_ratio"]]
    #print(mlb_df)
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q2: There should be 26 teams being analysed for MLB"

    return mlb_total #stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

,Metropolitan area,population,win_loss_ratio
0,Houston,6772470,0.635802
1,Toronto,5928040,0.450617
2,Atlanta,5789700,0.555556
3,Milwaukee,1572482,0.588957
4,St. Louis,2807002,0.543210
5,Chicago,9512999,0.483077
6,Phoenix,4661537,0.506173
7,Los Angeles,13310447,0.529231
8,San Francisco Bay Area,6657982,0.524691
9,Cleveland,2055612,0.561728


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [118]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation():
    nfl_df=pd.read_csv("assets/nfl.csv")
    #print(nfl_df)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #print(cities["NFL"])
    nfl_df=nfl_df[nfl_df.year == 2018]
    nfl_df = nfl_df[["team", "W", "L"]]
    #print(nfl_df.shape) #(30,3) 
    #print(nfl_df)
    
    nfl_df['team'] = nfl_df.team.apply(lambda team: re.findall("([\w\s]*)\*", team)[0] if '*' in team else team)
    #print(nfl_df.shape) #(30,11)
    #print(nfl_df)
    nfl_df['team'] = nfl_df.team.apply(lambda team: re.findall("([\w\s]*)\+", team)[0] if '+' in team else team)
    #print(nfl_df.shape) #(30,11)
    #print(nfl_df)
    cities['NFL'] = cities.NFL.apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if '[' in team else team)
    #print(cities['NFL'].shape) # (51,)
    #print(cities[["Metropolitan area","NFL"]])
    nfl_df["team"]=nfl_df['team'].replace({"Los Angeles Rams" : "Rams",
                                            "Los Angeles Chargers" : "Rams",
                                            "New York Giants":"Giants",   
                                            "New York Jets":"Giants",
                                            "San Francisco 49ers":"49ers",
                                            "Oakland Raiders":"49ers"})
    #print(nfl_df['team'])

    nfl_df['team_only'] = nfl_df.team.apply(lambda x:x.rsplit(None,1)[-1])
    #print(nfl_df.shape) #(40.4)
    #print(nfl_df['team_only'])

    nfl_df["team_only"]=nfl_df['team_only'].replace({"Rams" : "RamsChargers",
                                                    "Giants":"GiantsJets",
                                                    "49ers":"49ersRaiders"})
    #print(nfl_df['team_only'])  #there are 2 rams chargers, giants jets, 49ers raiders

    cities.rename(columns={"Population (2016 est.)[8]":"population", "NFL":"team_only"},inplace=True)
    #cities["team_only"]=cities['team_only'].replace({"Nets":"Knicks Nets"})
    #print(cities["team_only"])
    
    nfl_df=nfl_df[["team_only","W","L"]]
    #print(nfl_df.shape) #(40,3)
    #print(nfl_df.W)
    nfl_df=nfl_df[~nfl_df.W.str.contains('AFC')]
    nfl_df=nfl_df[~nfl_df.W.str.contains('NFC')]
    #print(nfl_df.shape) #(32,3)
    nfl_df.W=nfl_df.W.astype("float")
    nfl_df.L=nfl_df.L.astype("float")
    #print(nfl_df.shape) #(32,3)
    nfl_df=nfl_df.groupby(by="team_only")[["W","L"]].mean()
    #print(nfl_df.shape) #(29,2) + the index column

    nfl_df.reset_index(inplace=True)
    #print(nfl_df.shape) #(29,3)
    #print(nfl_df)
    merged_df=pd.merge(nfl_df,cities, how="inner",on="team_only")
    #print(merged_df.shape) #(29.8)

    merged_df.W=nfl_df.W.astype("float")
    merged_df.L=nfl_df.L.astype("float")
    merged_df['win_loss_ratio']=merged_df.W/(merged_df.W + merged_df.L)
    #print(merged_df['win_loss_ratio'].shape) #(29,)
    
    #raise NotImplementedError()
    
    population_by_region = pd.to_numeric(merged_df["population"],errors='coerce')# pass in metropolitan area population from cities
    #print("population_by_region :",population_by_region.shape) #(29,)
    win_loss_by_region = pd.to_numeric(merged_df["win_loss_ratio"],errors='coerce') # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    #print("win_loss_by_region ",win_loss_by_region.shape) #(29,)
    nfl_total= merged_df[["Metropolitan area","population","win_loss_ratio"]]
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q2: There should be 29 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

0.004922112149349407

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [156]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# clean the dataframes
def nhl_corr():
        nhl_param=nhl_correlation()
        #nhl_param.reset_index()
        nhl_param.rename(columns={"Population (2016 est.)[8]":"population","Ratio" : "W/L"})
        
        return #nhl_param
nhl_corr()
#----------------------------------------------------------------------------------
def nba_corr():
    nba_param=nba_correlation()
    nba_param.rename(columns={"win_loss_ratio":"W/L"},inplace=True)
    #population=nba_param["population"].astype(float)
    #w_L=nba_param["W/L"].astype(float)
    #nba_param=stats.pearsonr(population,w_L)[0]
    return nba_param
nba_corr()
#----------------------------------------------------------------------------------
def mlb_corr():
    mlb_param=mlb_correlation()
    mlb_param.rename(columns={"win_loss_ratio":"W/L"},inplace=True)
    return mlb_param
mlb_corr()
#----------------------------------------------------------------------------------
def nfl_corr():
    nfl_param=nfl_correlation()
    nfl_param.rename(columns={"win_loss_ratio":"W/L"},inplace=True)
    return nfl_param
nfl_corr()
#----------------------------------------------------------------------------------

def sports_team_performance():
    # YOUR CODE HERE
    nhl = nhl_corr()
    print(nhl)
    nba = nba_corr()
    mlb = mlb_corr()
    nfl = nfl_corr()

   
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

AttributeError: 'numpy.float64' object has no attribute 'rename'